In [12]:
import tensorflow as tf
import pandas as pd
import numpy as np
import tensorflow as tf

In [13]:
#step 1 load data
df = pd.read_csv('data.csv')
# removed the features we dont care about
df = df.drop(['index', 'price', 'sq_price'], axis=1)
# we only use the first 10 rows
df = df[:10]
df

,area,bathrooms
0,2104.0,3.0
1,1600.0,3.0
2,2400.0,3.0
3,1416.0,2.0
4,3000.0,4.0
5,1985.0,4.0
6,1534.0,3.0
7,1427.0,3.0
8,1380.0,3.0
9,1494.0,3.0


In [14]:
# step 2 - add labels
# 1 is food buy and 0 is bad buy
df.loc[:, ("y1")] = [1, 1, 1, 0, 0, 1, 0, 1, 1, 1] #put all numbers in one column named y1
# y2 is a negation of y1, opposite
df.loc[:, ("y2")] = df["y1"] == 0  
df.loc[:, ("y2")] = df["y2"].astype(int) # turn TRUE/FALSE to int
df

,area,bathrooms,y1,y2
0,2104.0,3.0,1,0
1,1600.0,3.0,1,0
2,2400.0,3.0,1,0
3,1416.0,2.0,0,1
4,3000.0,4.0,0,1
5,1985.0,4.0,1,0
6,1534.0,3.0,0,1
7,1427.0,3.0,1,0
8,1380.0,3.0,1,0
9,1494.0,3.0,1,0


In [15]:
# Step 3 - prepare data for tensorflow (tensors)
# tensors are a generic version of vectors en matrices
inputX = df.loc[:, ['area', 'bathrooms']].as_matrix()
inputY = df.loc[:, ["y1", "y2"]].as_matrix()
print("inputX:\n",inputX)
print("\ninputY:\n",inputY)

inputX:
 [[2.104e+03 3.000e+00]
 [1.600e+03 3.000e+00]
 [2.400e+03 3.000e+00]
 [1.416e+03 2.000e+00]
 [3.000e+03 4.000e+00]
 [1.985e+03 4.000e+00]
 [1.534e+03 3.000e+00]
 [1.427e+03 3.000e+00]
 [1.380e+03 3.000e+00]
 [1.494e+03 3.000e+00]]

inputY:
 [[1 0]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [1 0]]


/home/dan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/home/dan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.


In [16]:
#step 4 - write out our hypeparameters
learning_rate = 0.000001
training_epochs = 2000
display_step = 50
n_samples = inputY.size 

In [17]:
#step - create our computation grph/neural network
# placeholders are gateways for data into our computation graph
x = tf.placeholder(tf.float32, [None, 2])  # for feature input tensors, none means any number of examples

# create wights
# 2x2 matrix, that we'll keep updating through the trning process
W = tf.Variable(tf.zeros([2, 2]))
#
# add biases
b = tf.Variable(tf.zeros([2]))


# forward propagation
y_values = tf.add(tf.matmul(x, W), b) 

# activation function
# softmax
y = tf.nn.softmax(y_values) 


# matrix of labels
y_ = tf.placeholder(tf.float32, [None,2]) 

In [18]:
# cost function: mean squared error
cost = tf.reduce_sum(tf.pow(y_ - y, 2))/(2*n_samples) #  reduce_sum summation of the elements in the tensor

# Gradient descent
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

In [19]:
# Initialize variables and tensorflow session
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [9]:
for i in range(training_epochs):  
    sess.run(optimizer, feed_dict={x: inputX, y_: inputY}) # Take a gradient descent step using our inputs and labels
    
    if (i) % display_step == 0:
        cc = sess.run(cost, feed_dict={x: inputX, y_:inputY})
        print("Training step:", '%04d' % (i), "cost=", "{:.9f}".format(cc)) #, \"W=", sess.run(W), "b=", sess.run(b)

print ("Optimization Finished!")
training_cost = sess.run(cost, feed_dict={x: inputX, y_: inputY})
print ("Training cost=", training_cost, "W=", sess.run(W), "b=", sess.run(b), '\n')

Training step: 0000 cost= 0.114958666
Training step: 0050 cost= 0.109539948
Training step: 0100 cost= 0.109539881
Training step: 0150 cost= 0.109539807
Training step: 0200 cost= 0.109539740
Training step: 0250 cost= 0.109539665
Training step: 0300 cost= 0.109539606
Training step: 0350 cost= 0.109539531
Training step: 0400 cost= 0.109539464
Training step: 0450 cost= 0.109539405
Training step: 0500 cost= 0.109539330
Training step: 0550 cost= 0.109539263
Training step: 0600 cost= 0.109539188
Training step: 0650 cost= 0.109539129
Training step: 0700 cost= 0.109539054
Training step: 0750 cost= 0.109538995
Training step: 0800 cost= 0.109538913
Training step: 0850 cost= 0.109538853
Training step: 0900 cost= 0.109538786
Training step: 0950 cost= 0.109538712
Training step: 1000 cost= 0.109538652
Training step: 1050 cost= 0.109538570
Training step: 1100 cost= 0.109538510
Training step: 1150 cost= 0.109538451
Training step: 1200 cost= 0.109538391
Training step: 1250 cost= 0.109538309
Training ste

In [10]:
sess.run(y, feed_dict={x: inputX })

array([[0.7112522 , 0.2887478 ],
       [0.66498977, 0.33501023],
       [0.73657656, 0.26342347],
       [0.6471879 , 0.3528121 ],
       [0.78335613, 0.2166439 ],
       [0.70069474, 0.29930523],
       [0.6586633 , 0.34133676],
       [0.6482863 , 0.35171372],
       [0.6436828 , 0.35631716],
       [0.65480113, 0.3451989 ]], dtype=float32)

In [11]:
0.73644835+ 0.26355165

1.0